### Курсовая работа по курсу "Рекомендательные системы"

- Целевая метрика precision@5
- Бейзлайн решения - [MainRecommender](https://github.com/geangohn/recsys-tutorial/blob/master/src/recommenders.py)
- Сдаем ссылку на github с решением. На github должен быть файл recommendations.csv (user_id | [rec_1, rec_2, ...] с рекомендациями. rec_i - реальные id item-ов (из retail_train.csv)

In [125]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit import als

# Модель второго уровня
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV

import os, sys
module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

# Написанные нами функции
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender

In [126]:
data = pd.read_csv('../webinar_3/raw_data/retail_train.csv')
item_features = pd.read_csv('../webinar_3/raw_data/product.csv')
user_features = pd.read_csv('../webinar_3/raw_data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)


# Важна схема обучения и валидации!
# -- давние покупки -- | -- 6 недель -- | -- 3 недель -- 
# подобрать размер 2-ого датасета (6 недель) --> learning curve (зависимость метрики recall@k от размера датасета)
val_lvl_1_size_weeks = 6
val_lvl_2_size_weeks = 3

data_train_lvl_1 = data[data['week_no'] < data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)]
data_val_lvl_1 = data[(data['week_no'] >= data['week_no'].max() - (val_lvl_1_size_weeks + val_lvl_2_size_weeks)) &
                      (data['week_no'] < data['week_no'].max() - (val_lvl_2_size_weeks))]

data_train_lvl_2 = data_val_lvl_1.copy()  # Для наглядности. Далее мы добавим изменения, и они будут отличаться
data_val_lvl_2 = data[data['week_no'] >= data['week_no'].max() - val_lvl_2_size_weeks]

data_train_lvl_1.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [127]:
n_items_before = data_train_lvl_1['item_id'].nunique()

data_train_lvl_1 = prefilter_items(data_train_lvl_1, item_features=item_features, take_n_popular=5000)

n_items_after = data_train_lvl_1['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 83685 to 5001


#### MainRecommender

In [128]:
recommender = MainRecommender(data_train_lvl_1)

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/5001 [00:00<?, ?it/s]

In [129]:
result_lvl_1 = data_val_lvl_1.groupby('user_id')['item_id'].unique().reset_index()
result_lvl_1.columns=['user_id', 'actual']
result_lvl_1.head(2)

,user_id,actual
0,1,"[853529, 865456, 867607, 872137, 874905, 87524..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870..."


In [130]:
def do_recommend(model, func, n, k):
    
    func_dic = {
        'get_als_recommendations': model.get_als_recommendations,
        'get_own_recommendations': model.get_own_recommendations,
        'get_similar_items_recommendation': model.get_similar_items_recommendation,
        'get_similar_users_recommendation': model.get_similar_users_recommendation
    }
    
    try:
        result = func_dic[func](n, k)
    
    except (IndexError, ValueError) as e:
        
        result = recommender._extend_with_top_popular([], k)
    
    return result

In [131]:
%%time 

k = 100
result_lvl_1['als'] = result_lvl_1['user_id'].apply(lambda x: 
                                                    do_recommend(recommender, 'get_als_recommendations', x, k))
result_lvl_1.head(2)

CPU times: user 38.5 s, sys: 1.29 s, total: 39.8 s
Wall time: 21 s


,user_id,actual,als
0,1,"[853529, 865456, 867607, 872137, 874905, 87524...","[1037332, 5577022, 883616, 1096261, 1094924, 8..."
1,2,"[15830248, 838136, 839656, 861272, 866211, 870...","[5569230, 5569845, 866211, 916122, 6534480, 98..."


In [132]:
result_lvl_1.apply(lambda row: precision_at_k(row['als'], row['actual'], k=5), axis=1).mean()

0.11550603528319406

### 2-lvl model

In [133]:
users_lvl_2 = pd.DataFrame(data_train_lvl_2['user_id'].unique())
users_lvl_2.columns = ['user_id']

users_lvl_2.head(2)

,user_id
0,2070
1,2021


In [134]:
# только warm start
train_users = data_train_lvl_1['user_id'].unique()
users_lvl_2 = users_lvl_2[users_lvl_2['user_id'].isin(train_users)]

users_lvl_2['candidates'] = users_lvl_2['user_id'].apply(lambda x: recommender.get_own_recommendations(x, N=50))
users_lvl_2.head(4)

,user_id,candidates
0,2070,"[1105426, 1097350, 879194, 948640, 928263, 944..."
1,2021,"[950935, 1119454, 835578, 863762, 1019142, 102..."
2,1753,"[967041, 963686, 948640, 1057168, 942475, 9421..."
3,2120,"[5707857, 1029743, 1106523, 5569230, 916122, 8..."


In [135]:
s = users_lvl_2.apply(lambda x: pd.Series(x['candidates']), axis=1).stack().reset_index(level=1, drop=True)
s.name = 'item_id'

users_lvl_2 = users_lvl_2.drop('candidates', axis=1).join(s)
users_lvl_2['flag'] = 1

users_lvl_2.head(4)

,user_id,item_id,flag
0,2070,1105426,1
0,2070,1097350,1
0,2070,879194,1
0,2070,948640,1


In [136]:
targets_lvl_2 = data_train_lvl_2.copy()
targets_lvl_2['target'] = 1  # тут только покупки 

targets_lvl_2 = users_lvl_2.merge(targets_lvl_2, on=['user_id', 'item_id'], how='left')

targets_lvl_2['target'].fillna(0, inplace= True)
targets_lvl_2.drop('flag', axis=1, inplace=True)

In [137]:
targets_lvl_2['target'].mean()

0.10460593102333061

### XGBoost

In [138]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [139]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [140]:
targets_lvl_2 = targets_lvl_2.merge(item_features, on='item_id', how='left')
targets_lvl_2 = targets_lvl_2.merge(user_features, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,commodity_desc,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SANDWICHES,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,DOMESTIC WINE,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown


#### FE

In [141]:
# добавим средний чек по каждому пользователю

basket_sum = targets_lvl_2.groupby(['user_id', 'basket_id'], as_index=False).agg({'sales_value': 'sum'})
basket_mean = basket_sum.groupby(['user_id'], as_index=False).agg({'sales_value': 'mean'}).\
            rename(columns={'sales_value':'basket_mean'})

targets_lvl_2 = targets_lvl_2.merge(basket_mean, on='user_id', how='left')

targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,sub_commodity_desc,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,SANDWICHES - (COLD),,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,VALUE GLASS WINE,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28


In [142]:
# кол-во покупок в каждой категрии по пользователям

user_cat_sale_count = targets_lvl_2.groupby(['user_id', 'commodity_desc'], as_index=True).\
                        agg({'commodity_desc': 'count'}).\
                        rename(columns={'commodity_desc':'commodity_count'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(user_cat_sale_count, on=['user_id', 'commodity_desc'], how='left')
targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,curr_size_of_product,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28,3
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4 LTR,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28,1


In [143]:
# кол-во покупок категории в неделю

commodity_week_count = targets_lvl_2.groupby(['commodity_desc', 'week_no'], as_index=True).\
                        agg({'commodity_desc': 'count'}).\
                        rename(columns={'commodity_desc':'commodity_week_count'}).\
                        groupby(['commodity_desc']).agg({'commodity_week_count': 'mean'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(commodity_week_count, on='commodity_desc', how='left')
targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28,3,2.666667
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45-54,U,50-74K,Unknown,Unknown,1,None/Unknown,5.28,1,6.500000


In [144]:
# стоимость товара к средней стоимости по категории

commodity_mean_price = targets_lvl_2.groupby(['commodity_desc', 'item_id'], as_index=True).\
                        agg({'sales_value': 'mean'}).\
                        rename(columns={'sales_value':'item_price'}).\
                        groupby(['commodity_desc']).agg({'item_price': 'mean'}).reset_index()

targets_lvl_2 = targets_lvl_2.merge(commodity_mean_price, on='commodity_desc', how='left')
targets_lvl_2['price_to_cat'] = targets_lvl_2[['sales_value', 'item_price']].\
                    apply(lambda row: row['sales_value']/row['item_price'], axis=1)
targets_lvl_2.head(2)

,user_id,item_id,basket_id,day,quantity,sales_value,store_id,retail_disc,trans_time,week_no,...,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,basket_mean,commodity_count,commodity_week_count,item_price,price_to_cat
0,2070,1105426,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50-74K,Unknown,Unknown,1,None/Unknown,5.28,3,2.666667,6.196250,NaN
1,2070,1097350,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,50-74K,Unknown,Unknown,1,None/Unknown,5.28,1,6.500000,13.146875,NaN


In [145]:
X_train = targets_lvl_2.drop('target', axis=1)
y_train = targets_lvl_2[['target']]

In [146]:
item_features.columns, user_features.columns

(Index(['item_id', 'manufacturer', 'department', 'brand', 'commodity_desc',
        'sub_commodity_desc', 'curr_size_of_product'],
       dtype='object'),
 Index(['age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
        'hh_comp_desc', 'household_size_desc', 'kid_category_desc', 'user_id'],
       dtype='object'))

In [147]:
cat_feats = ['manufacturer', 'department', 'brand', 'commodity_desc',
       'sub_commodity_desc', 'curr_size_of_product', 'age_desc', 'marital_status_code', 'income_desc', 'homeowner_desc',
        'hh_comp_desc', 'household_size_desc', 'kid_category_desc']

X_train[cat_feats] = X_train[cat_feats].astype('category')

In [148]:
X_train = pd.get_dummies(X_train, columns=cat_feats, drop_first=True)

In [149]:
xgb_model = XGBClassifier(random_state=42)

In [150]:
parameters = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [2, 3, 4],
    'max_leaves': [2, 3],
    'objective': ['binary:logistic', 'rank:pairwise']
}
grid_search = GridSearchCV(estimator = xgb_model,
                           param_grid = parameters,
                           scoring = 'accuracy',
                           cv = 10,
                           n_jobs = -1)

In [ ]:
grid_search = grid_search.fit(X_train, y_train)

#### Test

In [77]:
data_test = pd.read_csv('data/retail_test1.csv')
data_test = prefilter_items(data=data_test, take_n_popular=5000, item_features=item_features)
data_test.head(5)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,1340,41652823310,664,912987,1,8.49,446,0.0,52,96,0.0,0.0,8.49
1,588,41652838477,664,1024426,1,6.29,388,0.0,8,96,0.0,0.0,6.29
3,1602,41665647035,664,999999,1,7.99,334,0.0,1741,96,0.0,0.0,7.99
5,1602,41665647035,664,999999,1,3.29,334,-1.2,1741,96,0.0,0.0,3.29
7,1602,41665647035,664,1044259,1,3.99,334,0.0,1741,96,0.0,0.0,3.99


In [80]:
data_test = data_test[data_test['user_id'].isin(data_train['user_id'].unique())]
# data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]
data_test

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc,price
0,1340,41652823310,664,912987,1,8.49,446,0.00,52,96,0.0,0.0,8.49
1,588,41652838477,664,1024426,1,6.29,388,0.00,8,96,0.0,0.0,6.29
3,1602,41665647035,664,999999,1,7.99,334,0.00,1741,96,0.0,0.0,7.99
5,1602,41665647035,664,999999,1,3.29,334,-1.20,1741,96,0.0,0.0,3.29
7,1602,41665647035,664,1044259,1,3.99,334,0.00,1741,96,0.0,0.0,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...
88716,1705,41944916893,684,6554202,1,3.00,421,-0.69,1341,98,0.0,0.0,3.00
88717,1705,41944916893,684,999999,1,4.99,421,0.00,1341,98,0.0,0.0,4.99
88718,1705,41944916893,684,6632695,1,4.99,421,0.00,1341,98,0.0,0.0,4.99
88724,98,41944918649,684,847241,1,2.99,421,0.00,1702,98,0.0,0.0,2.99


In [81]:
result_test = data_test.groupby('user_id')['item_id'].unique().reset_index()
result_test.columns=['user_id', 'actual']
result_test.head(2)

,user_id,actual
0,1,"[999999, 883616, 940947, 959219, 991024, 10049..."
1,2,"[999999, 866211, 879769, 885023, 899624, 94094..."


In [82]:
%%time 

k = 50
result_test['als'] = result_test['user_id'].apply(lambda x: 
                                    recommender.get_als_recommendations(x, k))

CPU times: user 7.98 s, sys: 187 ms, total: 8.17 s
Wall time: 4.6 s


In [83]:
precision_k = result_test['user_id'].apply(lambda x: 
                                    precision_at_k(result_test.loc[result['user_id']==x, 
                                                                          'als'].squeeze(), 
                                                   result_test.loc[result['user_id']==x, 
                                                                          'actual'].squeeze(), 
                                                   k=5)).mean()

precision_k

0.05090909090909091